This notebook provides a robust method for length estimation and computes the mean error on a synthetic dataset. 

## 1 - Dataset

### 1.1 - Create csv

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from data_utils import create_csv
from data_generator import DataGenerator
%load_ext autoreload
%autoreload 2

In [ ]:
dataset_name = 'blender_v3'
size = None # has to be less or equal to 5000
dataset_path = '/root/data/blender/blender_v3/'
datas = create_csv(dataset_name=dataset_name,
                   dataset_path=dataset_path,
                   datas_list=['depth_map', 'annotations', 'mask'])
datas_list = ['depth_map', 'annotations', 'mask_left']
data_generator = DataGenerator(dataframe=datas,
                          size=size,
                          dataset_name=dataset_name,
                          datas_list=datas_list,
                          target_list=['length', 'width', 'height', 'volume'])

### 1.2 - Visualize random inputs

In [ ]:
import random

In [ ]:
#ix = random.randint(0, len(data_generator))
ix = 3458
mask = data_generator[ix][0]['mask_left']
dmap = data_generator[ix][0]['depth_map']
mdepth = dmap * mask
gt_length = data_generator[ix][1]['length']
print('Ground truth length : {} for {}-th image in dataset'.format(gt_length, ix))
plt.imshow(mask)
plt.show()
plt.imshow(dmap)
plt.show()
plt.imshow(mdepth)
plt.colorbar()
plt.show()

In [ ]:
import numpy as np

In [ ]:
# # quick hack because the crop and mask function is not optimal
x, y = np.nonzero(mdepth>10)
for (i,j) in zip(x,y):
    mask[i,j] = 0

In [ ]:
plt.imshow(mask * mdepth)
plt.colorbar()

### 1.3 - Compute base line length 

In [ ]:
from utils import convert_to_world_point
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from lenght_estimator import pca_length_estimation

In [ ]:
baseline_length = pca_length_estimation(dmap=dmap, mask=mask, width_ratio=0.05)
print('Base line length : {}, Ground_truth length : {}'.format(baseline_length, gt_length))

In [ ]:
from sklearn import decomposition
from sklearn import datasets

In [ ]:
y, x = np.nonzero(mask)
wx, wy, wz = convert_to_world_point(x, y, dmap * 10)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(wx, wy, wz)

In [ ]:
fish_cloud = np.array([wx, wy, wz]).T
fish_cloud -= np.mean(fish_cloud, axis=0)
pca = decomposition.PCA(n_components=1)
pca.fit(fish_cloud)
X = pca.transform(fish_cloud)
plt.plot(X)

In [ ]:
X = np.sort(X, axis=0)

In [ ]:
X

In [ ]:
width = int(len(X) * 0.05)

In [ ]:
X_start = X[:width]
X_end = X[-width:]
norm = []
for i in range(len(X_start)):
    print(np.sqrt((X_start[i]-X_end[-(i+1)])**2))
    norm.append(np.sqrt((X_start[i]-X_end[-(i+1)])**2))

In [ ]:
pred_length = sum(norm) / len(norm)
pred_length, gt_length

### 1.4 - Length error on all dataset

In [ ]:
from tqdm import trange

In [ ]:
#errors = []
#for ix in trange(len(data_generator)):
#    mask = data_generator[ix][0]['mask_left']
#    dmap = data_generator[ix][0]['depth_map']
#    x, y = np.nonzero(mdepth>10)
#    for (i,j) in zip(x,y):
#        mask[i,j] = 0
#    mdepth = dmap * mask
#    gt_length = data_generator[ix][1]['length']
#    pred_length = pca_length_estimation(dmap=dmap, mask=mask, width_ratio=0.05)
#    errors.append(np.abs(pred_length - gt_length) / gt_length)

In [ ]:
#print('Mean error on dataset : {}'.format(sum(errors)/len(errors)))

### 2 - Mask perturbations


#### 2.1 - Mask erosion

In [ ]:
from utils import compute_segmentation_error
import numpy as np
import cv2

In [ ]:
lengths = []
errors = []
for s in range(1, 22):
    if s > 0:
        kernel = np.ones((s * 2, s))
        eroded_mask = cv2.erode(mask, kernel)
    else:
        eroded_mask = mask
    segmentation_error = compute_segmentation_error(eroded_mask, mask)
    errors.append(segmentation_error)
    pred_length = pca_length_estimation(dmap=dmap, mask=eroded_mask, width_ratio=0.1)
    if s == 1 or s == 21:
        plt.imshow(eroded_mask)
        plt.show()
        print('Pred length: {}, Segmentation error : {}'.format(pred_length, segmentation_error))
    relative_error = np.abs(pred_length - baseline_length) / baseline_length
    print('Mask error : {}, Length error : {}'.format(segmentation_error, relative_error*100))
    lengths.append(relative_error * 100)

In [ ]:
plt.plot(errors, lengths)
plt.xlabel('Segmentation error % (erosion)')
plt.ylabel('Length relative  % error')
plt.show()

#### 2.2 - Mask dilatation

In [ ]:
lengths = []
errors = []
for s in range(0, 22):
    if s > 0:
        kernel = np.ones((s, s*2))
        eroded_mask = cv2.dilate(mask, kernel)
    else:
        eroded_mask = mask
    segmentation_error = compute_segmentation_error(eroded_mask, mask)
    errors.append(segmentation_error)
    pred_length = pca_length_estimation(dmap=dmap, mask=eroded_mask, width_ratio=0.05)
    if s == 1 or s == 21:
        plt.imshow(eroded_mask)
        plt.show()
        print('Pred length: {}, Segmentation error : {}'.format(pred_length, segmentation_error))
    relative_error = np.abs(pred_length - baseline_length) / baseline_length
    print('Mask error : {}, Length error : {}'.format(segmentation_error, relative_error*100))
    lengths.append(relative_error * 100)

In [ ]:
plt.plot(errors, lengths)
plt.xlabel('Segmentation error % (dilatation)')
plt.ylabel('Length relative  % error')
plt.show()

## 3 - Depth map perturbation

In [ ]:
import copy

In [ ]:
lengths = []
errors = []
for s in range(0, 22):
    new_depth = copy.deepcopy(dmap)
    if s > 0:
        noise = np.zeros(new_depth.shape, np.uint8)
        cv2.randn(noise, np.array(0), np.ones(1) * s * 0.7)
        new_depth += noise * mask
    depth_relative_error = np.nanmean(np.abs((new_depth * mask) - mdepth) / mdepth)
    errors.append(depth_relative_error * 100)
    pred_length = pca_length_estimation(dmap=new_depth, mask=mask, width_ratio=0.05)
    if s == 0 or s == 21:
        plt.imshow(new_depth)
        plt.show()
        print('pred length : {}, ground truth: {}'.format(pred_length, baseline_length))
    relative_error = np.abs(pred_length-baseline_length) / baseline_length
    print('Depth error : {}, Length error : {}'.format(depth_relative_error * 100, relative_error * 100))
    lengths.append(relative_error * 100)

In [ ]:
plt.plot(errors, lengths)
plt.xlabel('Depth map error (noise)')
plt.ylabel('Length relative error')
plt.show()

## 4 - Depth map + mask perturbation

### 4.1 - Gaussian noise + erosion

In [ ]:
filter_with_dmap = False

In [ ]:
lengths = np.zeros((21, 21))
errors_mask = []
errors_depth = []
all_masks = []
all_depth = []

for size in range(1, 22):
    if size > 0:
        kernel = np.ones((size * 2, size))
        new_mask = cv2.erode(mask, kernel)
    else:
        new_mask = mask
            
    segmentation_error = compute_segmentation_error(new_mask, mask)
    errors_mask.append(segmentation_error)
    all_masks.append(new_mask)
    
for s in range(1, 22): 
    new_depth = copy.deepcopy(mdepth)
    if s > 0:
        # creat some noise
        noise = np.zeros(new_depth.shape, np.uint8)
        cv2.randn(noise, np.array(0), np.ones(1) * s * 0.7)
        new_depth += noise * mask
        
    all_depth.append(new_depth)
    depth_relative_error = np.nanmean(np.abs((new_depth * mask) - mdepth) / mdepth)
    errors_depth.append(depth_relative_error * 100)

for (i,new_mask) in enumerate(all_masks):
    for (j, new_depth) in enumerate(all_depth):
        if filter_with_dmap == True:
            x, y = np.nonzero((new_depth * new_mask)>10)
            for (m,n) in zip(x,y):
                new_mask[m,n] = 0
        pred_length = pca_length_estimation(dmap=new_depth, mask=new_mask, width_ratio=0.05)
        relative_error = np.abs(pred_length-baseline_length) / baseline_length
        print(i, j)
        print('Mask error : {}, Depth error : {}, Length errror : {}'.format(errors_mask[i], errors_depth[j], relative_error * 100))
        lengths[j, i] = relative_error*100

In [ ]:
import seaborn as sns

In [ ]:
df = pd.DataFrame(lengths.T, index=errors_mask, columns=errors_depth)
df.head()

In [ ]:
sns.heatmap(df)
ax.set_ylabel('mask error (erosion) %')
ax.set_xlabel('segmentation error %')

In [ ]:
target_error = 10
ix_to_keep = df.columns < 30

In [ ]:
ax = sns.heatmap(df[df.loc[:, ix_to_keep] < target_error])
ax.set_ylabel('segmentation error %')
ax.set_xlabel('depth error %')
ax.set_title = 'Filtered heat map : {}'.format(target_error)

### 4.2 - Gaussian noise + dilatation

In [ ]:
filter_with_dmap = False

In [ ]:
lengths = np.zeros((21, 21))
errors_mask = []
errors_depth = []
all_masks = []
all_depth = []

for size in range(1, 22):
    if size > 0:
        kernel = np.ones((size, size*2))
        new_mask = cv2.dilate(mask, kernel)
    else:
        new_mask = mask
            
    segmentation_error = compute_segmentation_error(new_mask, mask)
    errors_mask.append(segmentation_error)
    all_masks.append(new_mask)
    
for s in range(1, 22): 
    new_depth = copy.deepcopy(mdepth)
    if s > 0:
        # creat some noise
        noise = np.zeros(new_depth.shape, np.uint8)
        cv2.randn(noise, np.array(0), np.ones(1) * s * 0.7)
        new_depth += noise * mask
        
    all_depth.append(new_depth)
    depth_relative_error = np.nanmean(np.abs((new_depth * mask) - mdepth) / mdepth)
    errors_depth.append(depth_relative_error * 100)

for (i,new_mask) in enumerate(all_masks):
    for (j, new_depth) in enumerate(all_depth):
        if filter_with_dmap == True:
            x, y = np.nonzero((new_depth * new_mask)>10)
            for (m,n) in zip(x,y):
                new_mask[m,n] = 0
        pred_length = pca_length_estimation(dmap=new_depth, mask=new_mask, width_ratio=0.05)
        relative_error = np.abs(pred_length-baseline_length) / baseline_length
        print(i, j)
        print('Mask error : {}, Depth error : {}, Length errror : {}'.format(errors_mask[i], errors_depth[j], relative_error * 100))
        lengths[j, i] = relative_error*100

In [ ]:
df = pd.DataFrame(lengths.T, index=errors_mask, columns=errors_depth)
df.head()

In [ ]:
sns.heatmap(df)
ax.set_ylabel('mask error (dilatation) %')
ax.set_xlabel('segmentation error %')

In [ ]:
target_error = 20
ix_to_keep = df.columns < 30

In [ ]:
ax = sns.heatmap(df[df.loc[:, ix_to_keep] < target_error])
ax.set_ylabel('segmentation error %')
ax.set_xlabel('depth error %')
ax.set_title = 'Filtered heat map : {}'.format(target_error)

In [ ]:
mask_error = 10

In [ ]:
def find_insertion

In [ ]:
df.columns.insert(5, mask_error)

In [ ]:
df.reindex(columns=df.columns.insert(2, mask_error)).interpolate(axis=1)